In [4]:
import tensorflow as tf
from tensorflow import image as img
import os
import shutil, sys
import numpy as np
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
import keras
from keras.utils import plot_model
from keras.applications.mobilenet import MobileNet
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

In [3]:
from keras.models import load_model
model = load_model('keras_aug_4_15_12.h5')

In [4]:
input = Input(shape=(224,224,3))
#from keras.applications import VGG19
#conv_base = VGG19(weights='imagenet', include_top= False, input_shape=(224, 224, 3))
#x = conv_base(input)
x = model.get_layer("vgg19")(input)
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.3) (x)
x = Dense(256, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
x = Dense(32, activation = "relu")(x)
preds = Dense(2, activation='softmax') (x)

model= Model(input, preds)

for layer in model.layers:
    if layer.name == 'vgg19':
        layer.trainable = False
model.compile(optimizer=optimizers.RMSprop(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Model)                multiple                  20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
__________

In [6]:
model.layers[1].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 196, 196, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 196, 196, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 196, 196, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 98, 98, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 98, 98, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 98, 98, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 49, 49, 128)       0         
__________

In [7]:
image_size = 224
batch_size = 30
train2_dir = './train2'
val2_dir = './validation2'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=360,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator_2 = train_datagen.flow_from_directory(
                    train2_dir,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator_2 = validation_datagen.flow_from_directory(
                        val2_dir,
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 857 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [8]:
class_weight = {0: 2.67,
                1: 1.0}

In [9]:
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 26s - loss: 0.9496 - acc: 0.6096 - val_loss: 0.5498 - val_acc: 0.7500
Epoch 2/10
 - 23s - loss: 0.8032 - acc: 0.8077 - val_loss: 0.4482 - val_acc: 0.7833
Epoch 3/10
 - 24s - loss: 0.6836 - acc: 0.8388 - val_loss: 0.3889 - val_acc: 0.8667
Epoch 4/10
 - 23s - loss: 0.5658 - acc: 0.8632 - val_loss: 0.3509 - val_acc: 0.8500
Epoch 5/10
 - 24s - loss: 0.5063 - acc: 0.8864 - val_loss: 0.3239 - val_acc: 0.8500
Epoch 6/10
 - 24s - loss: 0.4748 - acc: 0.8833 - val_loss: 0.2972 - val_acc: 0.8500
Epoch 7/10
 - 24s - loss: 0.4010 - acc: 0.9037 - val_loss: 0.3538 - val_acc: 0.8833
Epoch 8/10
 - 24s - loss: 0.3953 - acc: 0.9040 - val_loss: 0.3399 - val_acc: 0.8667
Epoch 9/10
 - 24s - loss: 0.3698 - acc: 0.9046 - val_loss: 0.3320 - val_acc: 0.8500
Epoch 10/10
 - 23s - loss: 0.3558 - acc: 0.8960 - val_loss: 0.4118 - val_acc: 0.8667


In [10]:
for layer in model.layers:
    if layer.name == 'vgg19':
        layer.trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 26s - loss: 0.5563 - acc: 0.8445 - val_loss: 0.1998 - val_acc: 0.9333
Epoch 2/10
 - 24s - loss: 0.4284 - acc: 0.8764 - val_loss: 0.9136 - val_acc: 0.7667
Epoch 3/10
 - 24s - loss: 0.4337 - acc: 0.8850 - val_loss: 0.5915 - val_acc: 0.8000
Epoch 4/10
 - 23s - loss: 0.4154 - acc: 0.8902 - val_loss: 0.4810 - val_acc: 0.7667
Epoch 5/10
 - 23s - loss: 0.4547 - acc: 0.8657 - val_loss: 0.3365 - val_acc: 0.8833
Epoch 6/10
 - 24s - loss: 0.3971 - acc: 0.8770 - val_loss: 0.5552 - val_acc: 0.8667
Epoch 7/10
 - 24s - loss: 0.3965 - acc: 0.8933 - val_loss: 0.4600 - val_acc: 0.8667
Epoch 8/10
 - 25s - loss: 0.4831 - acc: 0.8680 - val_loss: 0.3473 - val_acc: 0.8667
Epoch 9/10
 - 24s - loss: 0.3671 - acc: 0.9017 - val_loss: 0.2362 - val_acc: 0.9167
Epoch 10/10
 - 23s - loss: 0.3973 - acc: 0.8773 - val_loss: 0.2859 - val_acc: 0.9000


In [11]:
model.compile(optimizer=optimizers.RMSprop(lr=0.000005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.3588 - acc: 0.8922 - val_loss: 0.2527 - val_acc: 0.8833
Epoch 2/10
 - 24s - loss: 0.3083 - acc: 0.9150 - val_loss: 0.2530 - val_acc: 0.9167
Epoch 3/10
 - 24s - loss: 0.2791 - acc: 0.9259 - val_loss: 0.5165 - val_acc: 0.8333
Epoch 4/10
 - 24s - loss: 0.2974 - acc: 0.9172 - val_loss: 0.3037 - val_acc: 0.8833
Epoch 5/10
 - 24s - loss: 0.3042 - acc: 0.9206 - val_loss: 0.3283 - val_acc: 0.9333
Epoch 6/10
 - 24s - loss: 0.2992 - acc: 0.9163 - val_loss: 0.7285 - val_acc: 0.7667
Epoch 7/10
 - 24s - loss: 0.2409 - acc: 0.9379 - val_loss: 0.2294 - val_acc: 0.9500
Epoch 8/10
 - 23s - loss: 0.2519 - acc: 0.9252 - val_loss: 0.7367 - val_acc: 0.8167
Epoch 9/10
 - 24s - loss: 0.2694 - acc: 0.9190 - val_loss: 0.4804 - val_acc: 0.8667
Epoch 10/10
 - 23s - loss: 0.2680 - acc: 0.9141 - val_loss: 0.1549 - val_acc: 0.9167


In [12]:
model.compile(optimizer=optimizers.RMSprop(lr=0.000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.2276 - acc: 0.9405 - val_loss: 0.3326 - val_acc: 0.8833
Epoch 2/10
 - 23s - loss: 0.2295 - acc: 0.9370 - val_loss: 0.3328 - val_acc: 0.9167
Epoch 3/10
 - 23s - loss: 0.1935 - acc: 0.9436 - val_loss: 0.3208 - val_acc: 0.9000
Epoch 4/10
 - 23s - loss: 0.2124 - acc: 0.9428 - val_loss: 0.3775 - val_acc: 0.8833
Epoch 5/10
 - 23s - loss: 0.1788 - acc: 0.9528 - val_loss: 0.3424 - val_acc: 0.9167
Epoch 6/10
 - 24s - loss: 0.2011 - acc: 0.9546 - val_loss: 0.3135 - val_acc: 0.9000
Epoch 7/10
 - 23s - loss: 0.2255 - acc: 0.9370 - val_loss: 0.2784 - val_acc: 0.9333
Epoch 8/10
 - 24s - loss: 0.1615 - acc: 0.9647 - val_loss: 0.5471 - val_acc: 0.8667
Epoch 9/10
 - 23s - loss: 0.2337 - acc: 0.9368 - val_loss: 0.2198 - val_acc: 0.9333
Epoch 10/10
 - 23s - loss: 0.1990 - acc: 0.9517 - val_loss: 0.2599 - val_acc: 0.9167


In [13]:
model.save('2class_224_best_1.h5')

In [14]:
model.compile(optimizer=optimizers.RMSprop(lr=0.0000005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.2409 - acc: 0.9356 - val_loss: 0.2503 - val_acc: 0.9333
Epoch 2/10
 - 24s - loss: 0.1772 - acc: 0.9543 - val_loss: 0.4239 - val_acc: 0.8833
Epoch 3/10
 - 24s - loss: 0.1747 - acc: 0.9520 - val_loss: 0.4446 - val_acc: 0.9000
Epoch 4/10
 - 23s - loss: 0.2079 - acc: 0.9486 - val_loss: 0.2387 - val_acc: 0.9333
Epoch 5/10
 - 25s - loss: 0.1669 - acc: 0.9505 - val_loss: 0.3670 - val_acc: 0.8833
Epoch 6/10
 - 22s - loss: 0.1875 - acc: 0.9535 - val_loss: 0.3536 - val_acc: 0.9167
Epoch 7/10
 - 23s - loss: 0.1960 - acc: 0.9477 - val_loss: 0.5908 - val_acc: 0.8667
Epoch 8/10
 - 25s - loss: 0.1931 - acc: 0.9420 - val_loss: 0.2998 - val_acc: 0.9333
Epoch 9/10
 - 23s - loss: 0.1989 - acc: 0.9413 - val_loss: 0.3460 - val_acc: 0.9167
Epoch 10/10
 - 24s - loss: 0.1664 - acc: 0.9471 - val_loss: 0.5670 - val_acc: 0.8667


In [8]:
model.compile(optimizer=optimizers.RMSprop(lr=0.000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.9283 - acc: 0.6287 - val_loss: 0.6132 - val_acc: 0.6833
Epoch 2/10
 - 23s - loss: 0.9124 - acc: 0.6809 - val_loss: 0.6125 - val_acc: 0.6833
Epoch 3/10
 - 23s - loss: 0.9241 - acc: 0.6659 - val_loss: 0.6154 - val_acc: 0.6833
Epoch 4/10
 - 24s - loss: 0.9126 - acc: 0.6655 - val_loss: 0.6213 - val_acc: 0.6667
Epoch 5/10
 - 23s - loss: 0.9146 - acc: 0.6944 - val_loss: 0.6149 - val_acc: 0.6833
Epoch 6/10
 - 24s - loss: 0.9189 - acc: 0.6654 - val_loss: 0.6158 - val_acc: 0.6833
Epoch 7/10
 - 23s - loss: 0.8825 - acc: 0.6858 - val_loss: 0.6180 - val_acc: 0.6667
Epoch 8/10
 - 23s - loss: 0.9184 - acc: 0.6712 - val_loss: 0.6149 - val_acc: 0.6833
Epoch 9/10
 - 24s - loss: 0.9177 - acc: 0.6617 - val_loss: 0.6192 - val_acc: 0.6667
Epoch 10/10
 - 23s - loss: 0.9273 - acc: 0.6706 - val_loss: 0.6156 - val_acc: 0.6667


In [12]:
for layer in model.layers:
    if layer.name == 'vgg19':
        layer.trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.8369 - acc: 0.7401 - val_loss: 0.6175 - val_acc: 0.7333
Epoch 2/10
 - 23s - loss: 0.7238 - acc: 0.7712 - val_loss: 0.4793 - val_acc: 0.7667
Epoch 3/10
 - 23s - loss: 0.8075 - acc: 0.7562 - val_loss: 0.6547 - val_acc: 0.6833
Epoch 4/10
 - 25s - loss: 0.7755 - acc: 0.7387 - val_loss: 0.5822 - val_acc: 0.7667
Epoch 5/10
 - 24s - loss: 0.7899 - acc: 0.7540 - val_loss: 0.5030 - val_acc: 0.8167
Epoch 6/10
 - 22s - loss: 0.7929 - acc: 0.7468 - val_loss: 0.6213 - val_acc: 0.7333
Epoch 7/10
 - 24s - loss: 0.7514 - acc: 0.7678 - val_loss: 0.5446 - val_acc: 0.7833
Epoch 8/10
 - 23s - loss: 0.7315 - acc: 0.7747 - val_loss: 0.5726 - val_acc: 0.7667
Epoch 9/10
 - 24s - loss: 0.7874 - acc: 0.7413 - val_loss: 0.5310 - val_acc: 0.7667
Epoch 10/10
 - 23s - loss: 0.7860 - acc: 0.7651 - val_loss: 0.5429 - val_acc: 0.7500


In [14]:
model.compile(optimizer=optimizers.RMSprop(lr=0.000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.7049 - acc: 0.7881 - val_loss: 0.5330 - val_acc: 0.7167
Epoch 2/10
 - 23s - loss: 0.6992 - acc: 0.7824 - val_loss: 0.5239 - val_acc: 0.7167
Epoch 3/10
 - 24s - loss: 0.7046 - acc: 0.7735 - val_loss: 0.5070 - val_acc: 0.7000
Epoch 4/10
 - 24s - loss: 0.6791 - acc: 0.8048 - val_loss: 0.5060 - val_acc: 0.7333
Epoch 5/10
 - 24s - loss: 0.6699 - acc: 0.7939 - val_loss: 0.5199 - val_acc: 0.6833
Epoch 6/10
 - 24s - loss: 0.6856 - acc: 0.7924 - val_loss: 0.5170 - val_acc: 0.7000
Epoch 7/10
 - 23s - loss: 0.6629 - acc: 0.7902 - val_loss: 0.5053 - val_acc: 0.7000
Epoch 8/10
 - 24s - loss: 0.7118 - acc: 0.7686 - val_loss: 0.4923 - val_acc: 0.6833
Epoch 9/10
 - 24s - loss: 0.6231 - acc: 0.8016 - val_loss: 0.5228 - val_acc: 0.7167
Epoch 10/10
 - 23s - loss: 0.6428 - acc: 0.7907 - val_loss: 0.5089 - val_acc: 0.7167


In [15]:
model.save('2class_224.h5')

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=0.000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=25,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/25
 - 26s - loss: 0.7111 - acc: 0.7867 - val_loss: 0.4856 - val_acc: 0.6833
Epoch 2/25
 - 24s - loss: 0.6619 - acc: 0.7931 - val_loss: 0.5019 - val_acc: 0.7000
Epoch 3/25
 - 24s - loss: 0.6734 - acc: 0.7962 - val_loss: 0.4814 - val_acc: 0.6833
Epoch 4/25
 - 24s - loss: 0.6494 - acc: 0.8011 - val_loss: 0.4742 - val_acc: 0.7000
Epoch 5/25
 - 23s - loss: 0.6788 - acc: 0.7970 - val_loss: 0.4734 - val_acc: 0.7000
Epoch 6/25
 - 23s - loss: 0.6386 - acc: 0.8039 - val_loss: 0.4844 - val_acc: 0.6667
Epoch 7/25
 - 23s - loss: 0.6485 - acc: 0.7853 - val_loss: 0.4786 - val_acc: 0.7167
Epoch 8/25
 - 23s - loss: 0.6154 - acc: 0.8146 - val_loss: 0.4681 - val_acc: 0.7000
Epoch 9/25


In [67]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2)

Epoch 1/10
 - 23s - loss: 0.5178 - acc: 0.8790 - val_loss: 0.6855 - val_acc: 0.7833
Epoch 2/10
 - 22s - loss: 0.2407 - acc: 0.9043 - val_loss: 1.1232 - val_acc: 0.7667
Epoch 3/10
 - 22s - loss: 0.2290 - acc: 0.9206 - val_loss: 0.6427 - val_acc: 0.8167
Epoch 4/10
 - 23s - loss: 0.2490 - acc: 0.9006 - val_loss: 0.7012 - val_acc: 0.8167
Epoch 5/10
 - 22s - loss: 0.2950 - acc: 0.8920 - val_loss: 0.4910 - val_acc: 0.7500
Epoch 6/10
 - 23s - loss: 0.2286 - acc: 0.9048 - val_loss: 0.4771 - val_acc: 0.8500
Epoch 7/10
 - 22s - loss: 0.2427 - acc: 0.9098 - val_loss: 0.6937 - val_acc: 0.7833
Epoch 8/10
 - 22s - loss: 0.2503 - acc: 0.8894 - val_loss: 0.4896 - val_acc: 0.8167
Epoch 9/10
 - 22s - loss: 0.2213 - acc: 0.9086 - val_loss: 0.4751 - val_acc: 0.8333
Epoch 10/10
 - 23s - loss: 0.2417 - acc: 0.9152 - val_loss: 0.4498 - val_acc: 0.8500


In [68]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2)

Epoch 1/10
 - 24s - loss: 0.1834 - acc: 0.9198 - val_loss: 0.5354 - val_acc: 0.8333
Epoch 2/10
 - 21s - loss: 0.1625 - acc: 0.9382 - val_loss: 0.5456 - val_acc: 0.8500
Epoch 3/10
 - 23s - loss: 0.1444 - acc: 0.9509 - val_loss: 0.6095 - val_acc: 0.8333
Epoch 4/10
 - 22s - loss: 0.1374 - acc: 0.9436 - val_loss: 0.6924 - val_acc: 0.8500
Epoch 5/10
 - 22s - loss: 0.1330 - acc: 0.9425 - val_loss: 0.5729 - val_acc: 0.8500
Epoch 6/10
 - 23s - loss: 0.1521 - acc: 0.9367 - val_loss: 0.6035 - val_acc: 0.8500
Epoch 7/10
 - 21s - loss: 0.1451 - acc: 0.9285 - val_loss: 0.6076 - val_acc: 0.8333
Epoch 8/10
 - 23s - loss: 0.1433 - acc: 0.9367 - val_loss: 0.6350 - val_acc: 0.8500
Epoch 9/10
 - 22s - loss: 0.1236 - acc: 0.9436 - val_loss: 0.7460 - val_acc: 0.8333
Epoch 10/10
 - 23s - loss: 0.1150 - acc: 0.9563 - val_loss: 0.6241 - val_acc: 0.8500


In [69]:
model.save('2clasifier_10_epochs.h5')

In [74]:
class_weight = {0: 2.67,
                1: 1.0}

In [78]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.0000005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.1960 - acc: 0.9509 - val_loss: 0.4084 - val_acc: 0.8500
Epoch 2/10
 - 21s - loss: 0.2374 - acc: 0.9482 - val_loss: 0.3395 - val_acc: 0.8500
Epoch 3/10
 - 23s - loss: 0.2188 - acc: 0.9452 - val_loss: 0.3588 - val_acc: 0.8500
Epoch 4/10
 - 23s - loss: 0.1931 - acc: 0.9609 - val_loss: 0.4080 - val_acc: 0.8833
Epoch 5/10
 - 20s - loss: 0.1630 - acc: 0.9540 - val_loss: 0.4406 - val_acc: 0.8833
Epoch 6/10
 - 24s - loss: 0.1872 - acc: 0.9463 - val_loss: 0.4480 - val_acc: 0.8833
Epoch 7/10
 - 22s - loss: 0.1577 - acc: 0.9609 - val_loss: 0.5272 - val_acc: 0.8667
Epoch 8/10
 - 21s - loss: 0.1788 - acc: 0.9555 - val_loss: 0.6430 - val_acc: 0.8500
Epoch 9/10
 - 23s - loss: 0.1986 - acc: 0.9535 - val_loss: 0.5387 - val_acc: 0.8833
Epoch 10/10
 - 22s - loss: 0.1601 - acc: 0.9597 - val_loss: 0.4772 - val_acc: 0.8667


In [79]:
model.save('2clasifier_30_epochs.h5')

In [80]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.0000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 24s - loss: 0.1571 - acc: 0.9503 - val_loss: 0.4847 - val_acc: 0.8833
Epoch 2/10
 - 21s - loss: 0.1935 - acc: 0.9535 - val_loss: 0.4803 - val_acc: 0.8833
Epoch 3/10
 - 23s - loss: 0.1981 - acc: 0.9436 - val_loss: 0.4762 - val_acc: 0.8833
Epoch 4/10
 - 21s - loss: 0.1820 - acc: 0.9586 - val_loss: 0.4723 - val_acc: 0.8833
Epoch 5/10
 - 22s - loss: 0.2414 - acc: 0.9517 - val_loss: 0.4412 - val_acc: 0.8833
Epoch 6/10
 - 22s - loss: 0.1995 - acc: 0.9451 - val_loss: 0.4400 - val_acc: 0.8833
Epoch 7/10
 - 21s - loss: 0.2180 - acc: 0.9486 - val_loss: 0.4173 - val_acc: 0.8833
Epoch 8/10
 - 23s - loss: 0.1838 - acc: 0.9546 - val_loss: 0.4131 - val_acc: 0.8833
Epoch 9/10
 - 23s - loss: 0.1973 - acc: 0.9463 - val_loss: 0.4031 - val_acc: 0.8833
Epoch 10/10
 - 21s - loss: 0.1722 - acc: 0.9515 - val_loss: 0.4135 - val_acc: 0.8833


In [81]:
model.save('2clasifier_40_epochs.h5')

In [82]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.00000005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 26s - loss: 0.1954 - acc: 0.9505 - val_loss: 0.4366 - val_acc: 0.8667
Epoch 2/10
 - 21s - loss: 0.1312 - acc: 0.9666 - val_loss: 0.4517 - val_acc: 0.8667
Epoch 3/10
 - 22s - loss: 0.1798 - acc: 0.9601 - val_loss: 0.4378 - val_acc: 0.8833
Epoch 4/10
 - 23s - loss: 0.1628 - acc: 0.9551 - val_loss: 0.4432 - val_acc: 0.8833
Epoch 5/10
 - 21s - loss: 0.1332 - acc: 0.9597 - val_loss: 0.4485 - val_acc: 0.8833
Epoch 6/10
 - 23s - loss: 0.1870 - acc: 0.9512 - val_loss: 0.4512 - val_acc: 0.8833
Epoch 7/10
 - 23s - loss: 0.1900 - acc: 0.9572 - val_loss: 0.4435 - val_acc: 0.8833
Epoch 8/10
 - 21s - loss: 0.1869 - acc: 0.9624 - val_loss: 0.4502 - val_acc: 0.8667
Epoch 9/10
 - 22s - loss: 0.2087 - acc: 0.9523 - val_loss: 0.4432 - val_acc: 0.8833
Epoch 10/10
 - 22s - loss: 0.2036 - acc: 0.9552 - val_loss: 0.4327 - val_acc: 0.8833


In [83]:
model.save('2clasifier_50_epochs.h5')

In [2]:
from keras.models import load_model
model = load_model('2class_224_best_1.h5')

In [8]:
val2_dir = './validation2'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        val2_dir,
        target_size=(224, 224),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

Found 60 images belonging to 2 classes.


In [9]:
predict_labels = np.argmax(predict, axis=1)
val_labels = np.zeros(predict_labels.size, dtype=int)
for i in range(2):
    for j in range(30):
        val_labels[i*30+j] = int(i);

In [10]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(val_labels, predict_labels)

array([[25,  5],
       [ 0, 30]])